# Используем алгоритм Витерби

Сначала попробуем на немецком языке

In [2]:
#импорт библиотек
import conllu 
from io import open
import pickle
import numpy as np

In [3]:
#парсим документ
doc_train = conllu.parse_incr(open("de_gsd-ud-train.conllu",'r',encoding = "utf-8"))

In [4]:
#С помощью словарей считаем количество n-граммов
c3 = {} # 3-граммов 
c2 = {} # биграммов
c1 = {} # монограммов
cT = {} # пар слово-тэг
for tokenlist in doc_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        #вернем 0, если триграмма нет в словаре 
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        #а если есть, увеличиваем счетчик на 1
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } )

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [5]:
#для удобства бросаем данные в pickle
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

with open('cs_german.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2686
267
19
43985


In [6]:
#возвращаем данные обратно
with open('cs_german.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [7]:
#делаем список уникальных тэгов
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
#счетчик уникальных тэгов
k = len(tags)

In [8]:
#вспомогательные функции из алгоритма Витерби
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)

#прогнозирует тэги в зависимости от положения слова в предложении
def K(n): 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['PROPN', 'PRON', 'PUNCT', 'VERB', 'NOUN', 'ADV', 'SYM', 'DET', 'PART', 'NUM', '_', 'AUX', 'ADJ', 'CCONJ', 'X', 'SCONJ', 'ADP']


Алгоритм Витерби в форме функции от токенизированного предложения

In [9]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))
    #возвращает список тэгов
    return y[1:]

Прогоняем на тестовом корпусе

In [10]:
doc_test = conllu.parse_incr(open("de_gsd-ud-train.conllu",'r',encoding = "utf-8"))
test_tags = [] # фактические тэги
predict_tags = [] # предсказанные тэги

for tokenlist in doc_test:
    # получаем предложение
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    # получаем оригинальные теги
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    # получаем предсказанные таги
    predict_tags.append(viterbi(sentence))

Оцениваем ошибку

In [11]:
err = 0.
for tag in range(len(test_tags)):
    err += np.mean(np.array(test_tags[tag]) != np.array(predict_tags[tag]))
print('Error = ', err/len(test_tags))

Error =  0.0353555743462


# Время усложнить задачу: посмотрим на работу алгоритма в рамках индонезийского. Шаги аналогичны.

In [13]:
#парсим документ
doc_train = conllu.parse_incr(open("id_gsd-ud-train.conllu",'r',encoding = "utf-8"))

In [15]:
#С помощью словарей считаем количество n-граммов
n3 = {} # 3-граммов 
n2 = {} # биграммов
n1 = {} # монограммов
nT = {} # пар слово-тэг
for tokenlist in doc_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        #вернем 0, если триграмма нет в словаре 
        n3old = n3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        #а если есть, увеличиваем счетчик на 1
        n3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): n3old+1 } )

    for i in range(0, len(sentence)-1):
        n2old = n2.get( (sentence[i][1], sentence[i+1][1]), 0)
        n2.update( { (sentence[i][1], sentence[i+1][1]): n2old+1 } )

    for i in range(0, len(sentence)):
        n1old = n1.get( (sentence[i][1]), 0)
        n1.update( { (sentence[i][1]): n1old+1 } )

    for i in range(0, len(sentence)):
        nTold = nT.get( (sentence[i][1], sentence[i][0]), 0)
        nT.update( { (sentence[i][1], sentence[i][0]): nTold+1 } )

In [16]:
#для удобства бросаем данные в pickle
print (len(n3))
print (len(n2))
print (len(n1))
print (len(nT))

with open('cs_id.pkl', 'wb') as f:
    pickle.dump([nT, n1, n2, n3], f)

2256
248
18
18981


In [17]:
#возвращаем данные обратно
with open('cs_id.pkl', 'rb') as f:
    nT, n1, n2, n3 = pickle.load(f)

In [18]:
#делаем список уникальных тэгов
tags_id = list(n1.keys())
tags_id.remove('*')
tags_id.remove('STOP')
#счетчик уникальных тэгов
k = len(tags_id)

In [19]:
doc_test = conllu.parse_incr(open("id_gsd-ud-train.conllu",'r',encoding = "utf-8"))
test_tags_id = [] # фактические тэги
predict_tags_id = [] # предсказанные тэги

for tokenlist in doc_test:
    # получаем предложение
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    # получаем оригинальные теги
    test_tags_id.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    # получаем предсказанные таги
    predict_tags_id.append(viterbi(sentence))

In [20]:
err = 0.
for tag in range(len(test_tags_id)):
    err += np.mean(np.array(test_tags_id[tag]) != np.array(predict_tags_id[tag]))
print('Error = ', err/len(test_tags_id))

Error =  0.848191920346


# Итоги

Имеем следующие результаты: на немецком корпусе ошибка составляет около 0.035, на индонезийском же ошибка значительно выше (около 0.85). Причиной тому можно считать специфичность и сложность индонезийского (отсутствие спряжений у глаголов, словообразование реализуется с помощью повторения слов, семья языка многочисленна). Алгоритм отрабатывает довольно долго (со второго раза нашлась машина, которая прогнала код). В будущем надеюсь найти оптимизированную библиотеку с реализацией алгори